# Publications markdown generator for academicpages

Takes a set of bibtex of publications and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

The core python code is also in `pubsFromBibs.py`. 
Run either from the `markdown_generator` folder after replacing updating the publist dictionary with:
* bib file names
* specific venue keys based on your bib file preferences
* any specific pre-text for specific files
* Collection Name (future feature)

TODO: Make this work with other databases of citations, 
TODO: Merge this with the existing TSV parsing solution

In [7]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex 
from time import strptime
import string
import html
import os
import re

In [8]:
import sys
sys.executable

'/Users/snirgaz/.julia/conda/3/bin/python'

In [9]:
#todo: incorporate different collection types rather than a catch all publications, requires other changes to template
publist = {
#     "proceeding": {
#         "file" : "proceedings.bib",
#         "venuekey": "booktitle",
#         "venue-pretext": "In the proceedings of ",
#         "collection" : {"name":"publications",
#                         "permalink":"/publication/"}
        
#     },
    "journal":{
        "file": "mypubs.bib",
        "venuekey" : "journal",
        "venue-pretext" : "",
        "collection" : {"name":"publications",
                        "permalink":"/publication/"}
    } 
}

In [10]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [12]:
cts=[]
for pubsource in publist:
    parser = bibtex.Parser()
    bibdata = parser.parse_file(publist[pubsource]["file"])
    entries_sorted=sorted(bibdata.entries,key=lambda x: bibdata.entries[x].fields["year"],reverse=True)
    #loop through the individual references in a given bibtex file
    for ind,bib_id in enumerate(entries_sorted):
        entry=bibdata.entries[bib_id]
        b=entry.fields
        pub_year = f'{b["year"]}'
        print(pub_year)
        print(ind)
        #todo: this hack for month and day needs some cleanup
        if "month" in b.keys(): 
            if(len(b["month"])<3):
                pub_month = "0"+b["month"]
                pub_month = pub_month[-2:]
            elif(b["month"] not in range(12)):
                tmnth = strptime(b["month"][:3],'%b').tm_mon   
                pub_month = "{:02d}".format(tmnth) 
            else:
                pub_month = str(b["month"])
        if "day" in b.keys(): 
            pub_day = str(b["day"])
        citation=""
        persons=entry.persons["author"]
        num_of_authors=len(persons)
        author=persons[0]
        citation = author.first()[0]+" "+author.last()[0]
        if num_of_authors==2:
            author=persons[1]
            citation = citation+" and "+author.first()[0]+" "+author.last()[0]
        for i in range(1,num_of_authors-1):
            author=persons[i]
            citation = citation+", "+author.first()[0]+" "+author.last()[0]
        print(citation)
        author=persons[-1]
        citation = citation+" and "+author.first()[0]+" "+author.last()[0]+". "
        url = b["url"]
        clean_title = b["title"].replace("{", "").replace("}","").replace("\\","")
        print(clean_title)
        citation = citation+"<a href=\""+url+"\">"+clean_title+".</a>"
        journal = entry.fields["journal"]
        citation = citation+"<em> "+journal+"</em> "
        if "arXiv" in journal:
            citation= citation+","+pub_year
        else:
            vol=b["volume"]
            if "number" in b.keys(): 
                num=b["number"]
            else:
                num=""
            pages=b["pages"]

            if "number" in b.keys():
                citation = citation+vol+"("+num+")"+":"+pages+","+pub_year
            else:
                citation= citation+vol+":"+pages+","+pub_year
        cts.append(citation)
        # field may not exist for a reference


2020
0
Pavel Volkov, Snir Gazit
Magnon Bose-Einstein condensation and superconductivity in a frustrated Kondo lattice
2020
1
Maxwell Block, Johannes Motruk, Snir Gazit, Michael Zaletel, Zeph Landau, Umesh Vazirani
Performance of the rigorous renormalization group for first order phase transitions and topological phases
2020
2
Frederik Nathan, Rongchun Ge, Snir Gazit, Mark Rudner
Quasiperiodic Floquet-Thouless energy pump
2020
3
Maxime Dupont, Snir Gazit
Evidence for deconfined $U(1)$ gauge theory at the transition between toric code and double semion
2020
4
Byungmin Kang, S. Parameswaran, Andrew Potter, Romain Vasseur
Superuniversality from disorder at two-dimensional topological phase transitions
2020
5
Maxime Dupont, Snir Gazit
From trivial to topological paramagnets: The case of $Z_2$ and $Z_2^3$ symmetries in two dimensions
2020
6
Yevgeny Slobodkin, Yotam Mazuz-Harpaz, Sivan Refaely-Abramson, Snir Gazit, Hadar Steinberg
Quantum Phase Transitions of Trilayer Excitons in Atomically T

<ipython-input-12-36dd9c6f70ba>:29: DeprecationWarning: first() is deprecated since 0.19: use Person.first_names instead
  citation = author.first()[0]+" "+author.last()[0]
<ipython-input-12-36dd9c6f70ba>:29: DeprecationWarning: last() is deprecated since 0.19: use Person.last_names instead
  citation = author.first()[0]+" "+author.last()[0]
<ipython-input-12-36dd9c6f70ba>:35: DeprecationWarning: first() is deprecated since 0.19: use Person.first_names instead
  citation = citation+", "+author.first()[0]+" "+author.last()[0]
<ipython-input-12-36dd9c6f70ba>:35: DeprecationWarning: last() is deprecated since 0.19: use Person.last_names instead
  citation = citation+", "+author.first()[0]+" "+author.last()[0]
<ipython-input-12-36dd9c6f70ba>:38: DeprecationWarning: first() is deprecated since 0.19: use Person.first_names instead
  citation = citation+" and "+author.first()[0]+" "+author.last()[0]+". "
<ipython-input-12-36dd9c6f70ba>:38: DeprecationWarning: last() is deprecated since 0.19: 

In [13]:
all_text="<ol>"
for c in cts:
    all_text+="<li>"+c+"</li>"
all_text+="</ol>"

In [14]:
all_text

'<ol><li>Pavel Volkov, Snir Gazit and Jedediah Pixley. <a href="https://www.pnas.org/content/117/34/20462">Magnon Bose-Einstein condensation and superconductivity in a frustrated Kondo lattice.</a><em> Proceedings of the National Academy of Sciences</em> 117(34):20462--20467,2020</li><li>Maxwell Block, Johannes Motruk, Snir Gazit, Michael Zaletel, Zeph Landau, Umesh Vazirani and Norman Yao. <a href="https://arxiv.org/abs/2010.15851">Performance of the rigorous renormalization group for first order phase transitions and topological phases.</a><em> arXiv e-prints</em> ,2020</li><li>Frederik Nathan, Rongchun Ge, Snir Gazit, Mark Rudner and Michael Kolodrubetz. <a href="https://arxiv.org/abs/2010.11485">Quasiperiodic Floquet-Thouless energy pump.</a><em> arXiv e-prints</em> ,2020</li><li>Maxime Dupont, Snir Gazit and Thomas Scaffidi. <a href="https://arxiv.org/abs/2008.06509">Evidence for deconfined $U(1)$ gauge theory at the transition between toric code and double semion.</a><em> arXiv e

In [20]:
bibdata = parser.parse_file(publist["journal"]["file"])


BibliographyDataError: repeated bibliograhpy entry: Volkov20462

In [8]:
bibdata.entries['Gazit2009'].fields["journal"]

'Opt. Express'

In [26]:
citation=""
entry=bibdata.entries['Gazit2018']
persons=entry.persons["author"]
num_of_authors=len(persons)
author=persons[0]
citation = author.first+" "+author.last
if num_of_authors==2:
    citation = citation+" and "+author.first+" "+author.last
for i in range(1,num_of_authors-1):
    author=persons[i]
    citation = citation+", "+author.first+" "+author.last
author=persons[-1]
citation = citation+" and "+author.first+" "+author.last+"."
b=entry.fields
url = b["url"]
clean_title = b["title"].replace("{", "").replace("}","").replace("\\","")
citation = citation+"<a href=\">"+url+"\""+clean_title+".</a>"
journal = entry.fields["journal"]
citation = citation+"<em> "+journal+"</em>"
vol=b["volume"]
num=b["number"]
pages=b["pages"]
pub_year = f'{b["year"]}'
#todo: this hack for month and day needs some cleanup
if "month" in b.keys(): 
    if(len(b["month"])<3):
        pub_month = "0"+b["month"]
        pub_month = pub_month[-2:]
    elif(b["month"] not in range(12)):
        tmnth = strptime(b["month"][:3],'%b').tm_mon   
        pub_month = "{:02d}".format(tmnth) 
    else:
        pub_month = str(b["month"])
if "day" in b.keys(): 
    pub_day = str(b["day"])
citation = citation+vol+"("+num+")"+":"+pages+pub_year

TypeError: unsupported operand type(s) for +: 'method' and 'str'

In [10]:
citation

'Snir Gazit, Fakher Assaad, Subir Sachdev, Ashvin Vishwanath and Chong Wang.<a href=">https://www.pnas.org/content/115/30/E6987"Confinement transition of Z$_2$ gauge theories coupled to massless fermions: Emergent quantum chromodynamics and SO(5) symmetry.</a><em> Proceedings of the National Academy of Sciences</em>115(30):E6987--E69952018'

In [84]:
vol

'27'

In [23]:
bibtex.Person("Gazit, Snir")

Person('Gazit, Snir')

In [24]:
?bibtex.Person